In [1]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF


class DoubleConvolution(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConvolution, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class UNet(nn.Module):
    def __init__(
        self,
        input_channels=3,
        output_channels=1,
        features=[64, 128, 256, 512],
    ):
        super(UNet, self).__init__()

        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Encoder
        for f in features:
            self.downs.append(DoubleConvolution(input_channels, f))
            input_channels = f

        # lower bottleneck layers
        self.bottleneck = DoubleConvolution(features[-1], features[-1] * 2)

        # Decoder
        for f in reversed(features):
            self.ups.append(
                nn.Sequential(
                    nn.Upsample(scale_factor=2),
                    nn.Conv2d(
                        in_channels=2 * f, out_channels=f, kernel_size=3, padding=1
                    ),
                )
            )
            self.ups.append(DoubleConvolution(2 * f, f))

        self.final_convolution = nn.Conv2d(
            in_channels=features[0],
            out_channels=output_channels,
            kernel_size=3,
            padding=1,
        )

    def forward(self, x):
        skip_connections = list()
        for module in self.downs:
            x = module(x)
            skip_connections.append(x)
            x = self.pool(x)

        skip_connections = skip_connections[::-1]  # reverse order

        x = self.bottleneck(x)

        for i in range(0, len(self.ups), 2):
            x = self.ups[i](x)
            skip_connection = skip_connections[i // 2]
            if skip_connection.shape != x.shape:
                x = TF.resize(
                    x,
                    size=skip_connection.shape[2:],
                    interpolation=TF.InterpolationMode.NEAREST,
                )
            x = torch.cat([skip_connection, x], dim=1)
            x = self.ups[i + 1](x)

        x = self.final_convolution(x)

        return x

In [3]:
from torch.utils.data import Dataset
import numpy as np
import os

import rasterio


class CloudDataset(Dataset):
    def __init__(self, images, labels, transform_image=None, transform_label=None):
        self.images = images
        self.labels = labels
        self.transform_image = transform_image
        self.transform_label = transform_label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx: int):
        image = rasterio.open(self.images[idx]).read()
        label = rasterio.open(self.labels[idx]).read()

        image = image.astype(np.float32)
        label = label.astype(np.float32)

        image = np.moveaxis(image, 0, -1)
        label = np.moveaxis(label, 0, -1)

        if self.transform_image:
            image = self.transform_image(image)
        if self.transform_label:
            label = self.transform_label(label)

        return image, label

In [4]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    train_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        print(f"Batch {batch_idx}/{len(loader)}")

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output, target)

        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    return train_loss / len(loader)

In [5]:
def test(model, loader, criterion, device):
    model.eval()
    test_loss = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            print(f"Batch {batch_idx}/{len(loader)}")

            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()

    return test_loss / len(loader)

In [6]:
def compute_dataset_info(train_images, train_labels, test_images, test_labels):
    dataset_info = {
        "num_positives": 0,
        "num_negatives": 0,
        "mean": [0.0] * 13,
        "std": [0.0] * 13,
    }

    for images, labels in [
        (train_images, train_labels),
        (test_images, test_labels),
    ]:
        for image, label in zip(images, labels):
            try:
                label = rasterio.open(label).read()
                label = np.moveaxis(label, 0, -1)
                label = label.astype(np.float32)
            except Exception as e:
                print(e)
                print(label)

                dataset_info["num_positives"] += np.sum(label)
                dataset_info["num_negatives"] += np.prod(label.shape) - np.sum(label)

            try:
                image = rasterio.open(image).read()
                image = np.moveaxis(image, 0, -1)
                image = image.astype(np.float32)
            except Exception as e:
                print(e)
                print(image)

                dataset_info["mean"] += np.mean(image, axis=(0, 1))
                dataset_info["std"] += np.std(image, axis=(0, 1))

    dataset_info["mean"] /= (
        len(train_images) + len(test_images)
    )
    dataset_info["std"] /= (
        len(train_images) + len(test_images)
    )

    return dataset_info

In [7]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
from torch.utils.data import DataLoader
from torchvision import transforms

SEED = 42
BATCH_SIZE = 4
NUM_EPOCHS = 2
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
NUM_WORKERS = 0 if os.name == "nt" else 2
DATA_DIR = os.path.join("drive", "MyDrive", "data")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")
DATASET_INFO_PATH = os.path.join(DATA_DIR, "dataset_info.pt")
CHECKPOINTS_DIR = "checkpoints"

os.makedirs(CHECKPOINTS_DIR, exist_ok=True)

torch.manual_seed(SEED)
np.random.seed(SEED)

train_images = []
train_labels = []
test_images = []
test_labels = []

for path in os.listdir(TRAIN_DIR):
    for img in os.listdir(os.path.join(TRAIN_DIR, path, "image")):
        if not ".ipynb_checkpoints" in img:
            train_images.append(os.path.join(TRAIN_DIR, path, "image", img))

train_labels = [path.replace("image", "label") for path in train_images]

for path in os.listdir(TEST_DIR):
    for img in os.listdir(os.path.join(TEST_DIR, path, "image")):
        if not ".ipynb_checkpoints" in img:
            test_images.append(os.path.join(TEST_DIR, path, "image", img))


test_labels = [path.replace("image", "label") for path in test_images]

for path in train_images + train_labels + test_images + test_labels:
    if not os.path.isfile(path):
        print(path)

if not os.path.exists(DATASET_INFO_PATH):
    dataset_info = compute_dataset_info(train_images, train_labels, test_images, test_labels)
    torch.save(dataset_info, DATASET_INFO_PATH)

dataset_info = torch.load(DATASET_INFO_PATH)

transform_image = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(dataset_info["mean"], dataset_info["std"]),
])
transform_label = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = CloudDataset(train_images, train_labels, transform_image, transform_label)
test_dataset = CloudDataset(test_images, test_labels, transform_image, transform_label)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

model = UNet(input_channels=13, output_channels=1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss(
    pos_weight=torch.tensor(dataset_info["num_negatives"] / dataset_info["num_positives"]).to(DEVICE)
)

print("Number of cloud pixels:", dataset_info["num_positives"])
print("Number of non-cloud pixels:", dataset_info["num_negatives"])

Number of cloud pixels: 576936248.0
Number of non-cloud pixels: 3763238152.0


In [ ]:
train_losses = []
test_losses = []

for epoch in range(NUM_EPOCHS):
    checkpoint_path = os.path.join("checkpoints", f"unet_epoch_{epoch}.pt")

    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        train_losses.append(checkpoint["train_loss"])
        test_losses.append(checkpoint["test_loss"])
    else:
        train_loss = train(model, train_loader, optimizer, criterion, DEVICE)
        test_loss = test(model, test_loader, criterion, DEVICE)

        checkpoint = {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_loss": train_loss,
            "test_loss": test_loss,
        }
        torch.save(checkpoint, checkpoint_path)

        train_losses.append(train_loss)
        test_losses.append(test_loss)

    print(f"Epoch: {epoch}, Train loss: {train_losses[-1]:.4f}, Test loss: {test_losses[-1]:.4f}")

/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Batch 0/7776
Batch 1/7776
Batch 2/7776
Batch 3/7776
Batch 4/7776
Batch 5/7776
Batch 6/7776
Batch 7/7776
Batch 8/7776
Batch 9/7776
Batch 10/7776
Batch 11/7776
Batch 12/7776
Batch 13/7776
Batch 14/7776
Batch 15/7776
Batch 16/7776
Batch 17/7776
Batch 18/7776
Batch 19/7776
Batch 20/7776
Batch 21/7776
Batch 22/7776
Batch 23/7776
Batch 24/7776
Batch 25/7776
Batch 26/7776
Batch 27/7776
Batch 28/7776
Batch 29/7776
Batch 30/7776
Batch 31/7776
Batch 32/7776
Batch 33/7776
Batch 34/7776
Batch 35/7776
Batch 36/7776
Batch 37/7776
Batch 38/7776
Batch 39/7776
Batch 40/7776
Batch 41/7776
Batch 42/7776
Batch 43/7776
Batch 44/7776
Batch 45/7776
Batch 46/7776
Batch 47/7776
Batch 48/7776
Batch 49/7776
Batch 50/7776
Batch 51/7776
Batch 52/7776
Batch 53/7776
Batch 54/7776
Batch 55/7776
Batch 56/7776
Batch 57/7776
Batch 58/7776
Batch 59/7776
Batch 60/7776
Batch 61/7776
Batch 62/7776
Batch 63/7776
Batch 64/7776
Batch 65/7776
Batch 66/7776
Batch 67/7776
Batch 68/7776
Batch 69/7776
Batch 70/7776
Batch 71/7776
Ba

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses, label="Train loss")
plt.plot(test_losses, label="Test loss")
plt.legend(loc="best")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
THRESHOLD = 0.5

def show_batch(model, loader, device):
    batch = next(iter(loader))
    data, target = batch
    data, target = data.to(device), target.to(device)

    model.eval()
    with torch.no_grad():
        output = model(data)
        output = torch.sigmoid(output)
        thresholded_output = (output > THRESHOLD).float()

    output = output.cpu().numpy()
    thresholded_output = thresholded_output.cpu().numpy()
    target = target.cpu().numpy()
    data = data.cpu().numpy()

    for i in range(len(output)):
        _, ax = plt.subplots(1, 4, figsize=(15, 5))
        ax[0].imshow(data[i, 3, :, :], cmap="gray")
        ax[1].imshow(target[i, 0, :, :], vmin=0, vmax=1, cmap="gray")
        ax[2].imshow(output[i, 0, :, :], vmin=0, vmax=1, cmap="gray")
        ax[3].imshow(thresholded_output[i, 0, :, :], vmin=0, vmax=1, cmap="gray")
        plt.show()

In [ ]:
show_batch(model, train_loader, DEVICE)

In [ ]:
show_batch(model, test_loader, DEVICE)